In [ ]:
%pwd
!pip install roboflow

from roboflow import Roboflow
import os

rf = Roboflow(api_key="<API KEY HERE>")
project = rf.workspace("robocup-9jdud").project("black-parts-panorama-image")
version = project.version(4)
dataset = version.download("sam2")

# rename dataset.location to "data"
os.rename(dataset.location, "/scr/user/farisfaiz/batch_connect/sys/jupyter/output/8c6f0e8e-3adc-4485-9aa3-dca2edc863b6/sam2.1_training/data")

In [ ]:
!git clone https://github.com/facebookresearch/sam2.git
!wget -O /scr/user/farisfaiz/batch_connect/sys/jupyter/output/8c6f0e8e-3adc-4485-9aa3-dca2edc863b6/sam2.1_training/sam2/sam2/configs/train.yaml 'https://drive.usercontent.google.com/download?id=11cmbxPPsYqFyWq87tmLgBAQ6OZgEhPG3'
%cd ./sam2_clonedrepo/
!pip install -e .[dev] -q
!cd ./checkpoints && ./download_ckpts.sh

In [ ]:
# Script to rename roboflow filenames to something SAM 2.1 compatible.
# It may be possible to remove this step by tweaking sam2/sam2/configs/train.yaml.
import os
import re

# Define the folder containing the files to rename
FOLDER = "/scr/user/farisfaiz/batch_connect/sys/jupyter/output/8c6f0e8e-3adc-4485-9aa3-dca2edc863b6/sam2.1_training/data/train"

def rename_files(folder):
    try:
        # Ensure the folder exists
        if not os.path.exists(folder):
            raise FileNotFoundError(f"Specified folder does not exist: {folder}")
        
        # Get the list of files
        files = os.listdir(folder)
        if not files:
            print(f"No files found in the folder: {folder}")
            return
        
        for filename in files:
            old_path = os.path.join(folder, filename)
            # Ensure it's a file and not a directory
            if not os.path.isfile(old_path):
                print(f"Skipping non-file item: {filename}")
                continue
            
            # Replace all except the last dot with underscores
            new_filename = filename.replace(".", "_", filename.count(".") - 1)
            
            # Add a numeric suffix if the filename doesn't already have one
            if not re.search(r"_\d+\.\w+$", new_filename):
                base, ext = os.path.splitext(new_filename)
                new_filename = f"{base}_1{ext}"
            
            new_path = os.path.join(folder, new_filename)
            
            # Rename the file and report the change
            os.rename(old_path, new_path)
            print(f"Renamed: {filename} -> {new_filename}")
    
    except Exception as e:
        print(f"An error occurred: {e}")

# Call the function
rename_files(FOLDER)

In [ ]:
%cd sam2
%ls
!python training/train.py -c 'configs/train.yaml' --use-cluster 0 --num-gpus 2

In [ ]:
%load_ext tensorboard
%tensorboard --bind_all --logdir ./sam2_logs/

In [ ]:
%cd ../..
%pip show sam2
!pip install supervision -q

import torch
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
import supervision as sv
import os
import random
from PIL import Image
import numpy as np
import json
from skimage.draw import polygon  # Added for mask generation

# use bfloat16 for the entire notebook
# from Meta notebook
torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
if torch.cuda.get_device_properties(0).major >= 8:
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

parent_directory = "/scr/user/farisfaiz/batch_connect/sys/jupyter/output/8c6f0e8e-3adc-4485-9aa3-dca2edc863b6/sam2.1_training"

checkpoint = parent_directory + "/sam2_clonedrepo/sam2_logs/configs/train.yaml/checkpoints/checkpoint.pt"
model_cfg = "configs/sam2.1/sam2.1_hiera_b+.yaml"
#model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"
sam2 = build_sam2(model_cfg, checkpoint, device="cuda")
mask_generator = SAM2AutomaticMaskGenerator(sam2)

checkpoint_base = parent_directory + "/sam2_clonedrepo/checkpoints/sam2.1_hiera_base_plus.pt"
#model_cfg_base = "configs/sam2.1/sam2.1_hiera_l.yaml"
model_cfg_base = "configs/sam2.1/sam2.1_hiera_b+.yaml"
sam2_base = build_sam2(model_cfg_base, checkpoint_base, device="cuda")
mask_generator_base = SAM2AutomaticMaskGenerator(sam2_base)

In [ ]:
# Function to calculate Dice coefficient
def dice_coefficient(pred_mask, gt_mask):
    intersection = np.logical_and(pred_mask, gt_mask).sum()
    union = pred_mask.sum() + gt_mask.sum()
    if union == 0:
        return 1.0
    return 2 * intersection / union

# Function to calculate IoU (Intersection over Union)
def iou_score(pred_mask, gt_mask):
    intersection = np.logical_and(pred_mask, gt_mask).sum()
    union = np.logical_or(pred_mask, gt_mask).sum()
    if union == 0:
        return 1.0
    return intersection / union

# Function to evaluate model performance
def evaluate_model(mask_generator, image_path, gt_path):
    # Load image and generate predictions
    image = np.array(Image.open(image_path).convert("RGB"))
    result = mask_generator.generate(image)
    
    # Load ground truth masks from JSON
    with open(gt_path, 'r') as f:
        gt_data = json.load(f)
    
    # Convert ground truth annotations to masks
    height, width = image.shape[:2]
    gt_masks = []
    for annotation in gt_data['annotations']:
        mask = np.zeros((height, width), dtype=bool)
        # Convert segmentation polygon to mask
        for segment in annotation['segmentation']:
            poly = np.array(segment).reshape(-1, 2)
            rr, cc = polygon(poly[:, 1], poly[:, 0], shape=(height, width))
            mask[rr, cc] = True
        gt_masks.append(mask)
    
    # Calculate metrics for each predicted mask against all ground truth masks
    dice_scores = []
    iou_scores = []
    
    for pred_mask in result:
        pred_binary = pred_mask['segmentation']
        # Calculate best score against all ground truth masks
        max_dice = max(dice_coefficient(pred_binary, gt) for gt in gt_masks)
        max_iou = max(iou_score(pred_binary, gt) for gt in gt_masks)
        dice_scores.append(max_dice)
        iou_scores.append(max_iou)
    
    # Calculate mean scores
    mDice = np.mean(dice_scores) if dice_scores else 0
    mIoU = np.mean(iou_scores) if iou_scores else 0
    
    return mDice, mIoU

In [ ]:
validation_set = os.listdir(parent_directory+"/data/valid")

# Evaluate on validation set
total_dice = 0
total_iou = 0
count = 0

for img_file in [f for f in validation_set if f.endswith('.jpg')]:
    img_path = os.path.join(parent_directory + "/data/valid", img_file)
    gt_path = img_path.replace('.jpg', '.json')
    
    if os.path.exists(gt_path):
        dice, iou = evaluate_model(mask_generator, img_path, gt_path)
        total_dice += dice
        total_iou += iou
        count += 1
        print(f"Image {img_file}: mDice = {dice:.4f}, mIoU = {iou:.4f}")

if count > 0:
    print(f"\nOverall Metrics:")
    print(f"Mean Dice Score: {total_dice/count:.4f}")
    print(f"Mean IoU Score: {total_iou/count:.4f}")

In [ ]:
# Original visualization code
image = random.choice([img for img in validation_set if img.endswith(".jpg")])
image = os.path.join(parent_directory + "/data/valid", image)
opened_image = np.array(Image.open(image).convert("RGB"))
result = mask_generator.generate(opened_image)

detections = sv.Detections.from_sam(sam_result=result)

mask_annotator = sv.MaskAnnotator(color_lookup = sv.ColorLookup.INDEX)
annotated_image = opened_image.copy()
annotated_image = mask_annotator.annotate(annotated_image, detections=detections)

base_annotator = sv.MaskAnnotator(color_lookup = sv.ColorLookup.INDEX)

base_result = mask_generator_base.generate(opened_image)
base_detections = sv.Detections.from_sam(sam_result=base_result)
base_annotated_image = opened_image.copy()
base_annotated_image = base_annotator.annotate(base_annotated_image, detections=base_detections)

sv.plot_images_grid(images=[annotated_image, base_annotated_image], titles=["Fine-Tuned SAM-2.1", "Base SAM-2.1"], grid_size=(1, 2))

In [ ]:
image = random.choice([img for img in validation_set if img.endswith(".jpg")])
image = os.path.join(parent_directory + "/data/valid", image)
opened_image = np.array(Image.open(image).convert("RGB"))

import matplotlib.pyplot as plt
imgplot = plt.imshow(opened_image)
plt.show()

In [ ]:
result = mask_generator.generate(opened_image)
detections = sv.Detections.from_sam(sam_result=result)
mask_annotator = sv.MaskAnnotator(color_lookup = sv.ColorLookup.INDEX)
annotated_image = opened_image.copy()
annotated_image = mask_annotator.annotate(annotated_image, detections=detections)

masked_plot = plt.imshow(annotated_image)
plt.show()

In [ ]:
sv.plot_images_grid(images=[annotated_image, opened_image], titles=["Segmented Image", "Unsegmented Image"], grid_size=(1, 2))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

axes[0].imshow(annotated_image)
axes[0].set_title('Segmented Image')
axes[0].axis('off')  # Hide axes for clarity

axes[1].imshow(opened_image)
axes[1].set_title('Unsegmented Image')
axes[1].axis('off')  # Hide axes for clarity

plt.tight_layout()
plt.show()